In [188]:
import polars as pl

### Expressions for defining columns

In [189]:
def payer_category():
    return (
        pl.when(pl.col('payer_name') == 'Gross_Charge').then('gross')
        .when(pl.col('payer_name') == 'Cash_Charge').then('cash')
        .when(pl.col('payer_name') == 'Min_Negotiated_Rate').then('min')
        .when(pl.col('payer_name') == 'Max_Negotiated_Rate').then('max')
        .otherwise('payer').alias('payer_category')
    )

def hcpcs_cpt_and_drg():
    
    hcpcs_cpt = pl.when(pl.col('line_type') == '2-CPT').then(pl.col('code')).alias('hcpcs_cpt')
    ms_drg = pl.when(pl.col('line_type') == '3-DRG').then(pl.col('code')).str.zfill(3).alias('ms_drg')
    
    return hcpcs_cpt, ms_drg

def all_charges():
    
    standard_charge_percent = (
        pl.col('standard_charge')
        .str.extract('(\d+)% of gross charges')
        .alias('standard_charge_percent')
    )
    
    contracting_method = (
        pl.when(pl.col('standard_charge').str.contains('\% of gross charges'))
        .then('percent of total billed charge')
        .alias('contracting_method')
    )
    
    standard_charge = (
        pl.when(pl.col('standard_charge').str.contains('\% of gross charges'))
        .then(None).otherwise(pl.col('standard_charge')).alias('standard_charge')
    )

    return standard_charge, standard_charge_percent, contracting_method

In [190]:
file = '474063406_Ascension Saint Thomas Highlands_standardcharges.xlsx'

# These are things I learned from trial and error working with these files
# and also examining them in Numbers
read_csv_options = {
    'skip_rows': 1, 
    'infer_schema_length': 0, 
    'null_values': ['N/A']
}

### Trying a single file: an easy and fast workflow

My workflow is to build a single parenned object like
```py
(
    df
    .transformation
    .transformation
    ...
)
```
and then keep adding transformations until I get the dataframe I like. I do this in a single cell.

So I started like

```py
(
    pl.read_excel(file, sheet_id = 2, read_csv_options = read_csv_options)
)
```
and noticed that I could slice off the first 3 rows, so then I did
```py
(
    pl.read_excel(file, sheet_id = 2, read_csv_options = read_csv_options)
    .slice(3,)
)
```
and so on. What I ended up with was this:

In [191]:
(
    pl.read_excel(file, sheet_id = 2, read_csv_options = read_csv_options)
    .slice(3,)
    .drop(['Facility_BU_ID', 'UB_Revenue_Description'])
    .rename({
        'Code_Type':'line_type',
        'Code':'code',
        'Description':'description',
        'UB_Revenue_Code':'rev_code',
    })
    .melt(
        ['line_type', 'code', 'description', 'rev_code'],
        variable_name = 'payer_name',
        value_name = 'standard_charge',
    )
    .filter(pl.col('standard_charge').is_not_null())
    .with_columns([
        payer_category(),
        pl.col('rev_code').str.zfill(4).keep_name(),
        *hcpcs_cpt_and_drg(),
        *all_charges(),
    ])
).sample(20)

line_type,code,description,rev_code,payer_name,standard_charge,payer_category,hcpcs_cpt,ms_drg,standard_charge_percent,contracting_method
str,str,str,str,str,str,str,str,str,str,str
"""1-CDM""","""30013417""","""PEG DRILL INBO…","""0272""","""Max_Negotiated…","""446.52""","""max""",null,null,null,null
"""3-DRG""","""674""","""Other Kidney A…",null,"""H85_HUMANA_CHO…","""34625.3""","""payer""",null,"""674""",null,null
"""2-CPT""","""86603""","""ANTIBODY ADENO…",null,"""Max_Negotiated…","""21.54""","""max""","""86603""",null,null,null
"""2-CPT""","""90870""","""ELECTROCONVULS…",null,"""Max_Negotiated…","""172.65""","""max""","""90870""",null,null,null
"""3-DRG""","""453""","""Combined Anter…",null,"""H83_HUMANA_MED…","""69087.33""","""payer""",null,"""453""",null,null
"""2-CPT""","""92273""","""FULL FIELD ERG…",null,"""Min_Negotiated…","""76.84""","""min""","""92273""",null,null,null
"""2-CPT""","""15630""","""DELAY FLAP/SCT…",null,"""Max_Negotiated…","""1598.83""","""max""","""15630""",null,null,null
"""1-CDM""","""42106119""","""XR SHOULDER 1 …","""0320""","""A09_AETNA_2005…","""223.6""","""payer""",null,null,null,null
"""1-CDM""","""1656775""","""ZIPRASIDONE 10…","""0636""","""C04_CIGNA_PPO_…","""110.56""","""payer""",null,null,null,null


Let's turn this into a function so that we can process any excel file (assuming they're formatted the same way. We'll inspect this later.

In [192]:
def process(url):
    
    ### Putting my notes here as I go:
    
    ### Around the 20th file in: upon trying to cast 
    ### 'standard_charge' to float, I got:
    
    ### ComputeError: strict conversion from `str` to `f64` failed for value(s) 
    ### ["85% of gross charges", "65% of gross charges", … "82% of gross charges"]; 
    ### if you were trying to cast Utf8 to temporal dtypes, consider using `strptime`
    
    ### So this tells me that this a relatively uncommon thing, but that it does happen
    
    print(url)    
                
    df = (
        pl.read_excel(url, sheet_id = 2, read_csv_options = read_csv_options)
        .slice(3,)
        .drop(['Facility_BU_ID', 'UB_Revenue_Description'])
        .rename({
            'Code_Type':'line_type',
            'Code':'code',
            'Description':'description',
            'UB_Revenue_Code':'rev_code',
        })
        .melt(
            ['line_type', 'code', 'description', 'rev_code'],
            variable_name = 'payer_name',
            value_name = 'standard_charge',
        )
        .filter(pl.col('standard_charge').is_not_null())
        .with_columns([
            payer_category(),
            pl.col('rev_code').str.zfill(4).keep_name(),
            *hcpcs_cpt_and_drg(),
            *all_charges(),
        ])
    )
    
    return df

### Getting the MRF links from the Ascension page

In [193]:
from bs4 import BeautifulSoup
import requests

headers = {
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36"
}

mrf_page = 'https://healthcare.ascension.org/price-transparency/price-transparency-files'

resp = requests.get(mrf_page, headers = headers)

In [194]:
soup = BeautifulSoup(resp.content)
mrfs = []
base_url = 'https://healthcare.ascension.org'
for link in soup.find_all('a'):
    if '.xlsx' in link.get('href'):
        mrfs.append((base_url + link.get('href'), link.text))
        
print(f'The number of MRFs found was {len(mrfs)}')

The number of MRFs found was 116


Here's an example of what's in the list of MRF tuples: it's just the link to the MRF, plus the hospital name as shown on the Ascension website.

In [195]:
mrfs[12]

('https://healthcare.ascension.org/-/media/project/ascension/healthcare/price-transparency-files/fl/300577249_ascension-sacred-heart-gulf_standardcharges.xlsx',
 'Ascension Sacred Heart Gulf')

### Looping through all the Ascension MRFs

In [196]:
from tqdm import tqdm
from glob import glob
import os

In [ ]:
for mrf_url, hospital_name in tqdm(mrfs):
    # Process each Excel file and save the processed data
    # in a .parquet file
    
    if not os.path.exists('data'):
        os.mkdir('data')
        
    basename = os.path.basename(mrf_url)
    
    path = f'data/{basename}'
    filename, ext = os.path.splitext(basename)
    
    parquet_name = f'{filename}.parquet'
    
    # Ascension won't serve you the binary unless you have the
    # right headers. So it's necessary to download the file with
    # requests and then open it up in a dataframe library.
    # This is fine, since we can then cache the files for later.
    
    if parquet_name in glob('data/*'):
        # We've already processed the file
        continue
    
    if path not in glob('data/*'):
        resp = requests.get(mrf_url, headers = headers)
        output = open(path, 'wb')
        output.write(resp.content)
        output.close()

    try:
        df = process(path)    
        df.write_parquet(f'data/{parquet_name}')
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"The problematic url was: {mrf_url}")

  0%|                                                                                                                                                     | 0/116 [00:00<?, ?it/s]

data/630578923_ascension-saint-vincents-east_standardcharges.xlsx


  1%|█▏                                                                                                                                           | 1/116 [00:12<23:06, 12.05s/it]

630578923_ascension-saint-vincents-east_standardcharges.parquet
data/630909073_saint-vincents-blount_standardcharges.xlsx


  2%|██▍                                                                                                                                          | 2/116 [00:17<15:51,  8.35s/it]

...

382377821_ascension-genesys-hospital_standardcharges.parquet


Exception ignored in: <function Xlsx2csv.__del__ at 0x7f3ae9e774c0>
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.9/site-packages/xlsx2csv.py", line 219, in __del__
    self.ziphandle.close()
AttributeError: 'Xlsx2csv' object has no attribute 'ziphandle'
 54%|████████████████████████████████████████████████████████████████████████████                                                                | 63/116 [17:38<11:40, 13.22s/it]

data/381360526_ascension-borgess-hospital-and-ascension-borgess-pipp-hospital_standardcharges.xlsx
An error occurred: Invalid xlsx file: data/381360526_ascension-borgess-hospital-and-ascension-borgess-pipp-hospital_standardcharges.xlsx
The problematic url was: https://healthcare.ascension.org/-/media/project/ascension/healthcare/price-transparency-files/mi/381360526_ascension-borgess-hospital-and-ascension-borgess-pipp-hospital_standardcharges.xlsx
data/381359180_ascension-borgess-allegan-hospital_standardcharges.xlsx


 55%|█████████████████████████████████████████████████████████████████████████████▏                                                              | 64/116 [17:39<08:27,  9.77s/it]

An error occurred: Error: potential invalid date format.
The problematic url was: https://healthcare.ascension.org/-/media/project/ascension/healthcare/price-transparency-files/mi/381359180_ascension-borgess-allegan-hospital_standardcharges.xlsx
data/381490190_ascension-borgess-lee-hospital_standardcharges.xlsx


In [ ]:
dfs

### Some problematic files:

* 272192831_ascension-st-vincent-dunn_standardcharges.xlsx doesn't have a Facility_BU_ID column (everything else is the same)
* ascension-medical-group-st-vincent-urgent-care.xlsx doesn't match the schema at all

### A script for matching names to CCNs